In [ ]:
%pylab inline
%run bizhawk.py
%run helper.py
import subprocess
from keras.applications.inception_v3 import InceptionV3
from sklearn.manifold import TSNE
import os

# Attract Mode

In [ ]:
rom_path = ''
bizhawk_path = 'BizHawk-2.2.2/'
lua_path = '../action.lua'

FRAMES_PER_STEP = 30
steps = 1200
ACTION_NUM = 4096 #There are in total 4096 combinations of actions
distribution = np.load('SNES_distribution.npy') #The distribution of button pressed
embedded_model = InceptionV3() #It takes a while to load the inception model

In [ ]:
embeddings = []

with BizHawk(rom_path, bizhawk_path, lua_path=lua_path) as bh:
    bh.send(b'client.speedmode(400);')
    for i in range(steps * FRAMES_PER_STEP):
        if i % FRAMES_PER_STEP == 0:
            temp_img = bh.read_img('temp.png')
            temp_img = np.expand_dims(temp_img, axis=0)
            temp_img = embedded_model.predict(temp_img)[0]
            embeddings.append(temp_img)
        bh.send(b'emu.frameadvance();')

attract_embeddings = np.array(embeddings)

# Chaos Monkey

In [ ]:
embeddings = []

#SNES actions
button_names = ['Up', 'Down', 'Left', 'Right', 'Select',
                'Start', 'B', 'A', 'X', 'Y', 'L', 'R']

with BizHawk(rom_path, bizhawk_path, lua_path=lua_path) as bh:
    bh.send(b'client.speedmode(400);')
    
    for i in range(steps * FRAMES_PER_STEP):
        code = b'buttons = {};'
        if i % FRAMES_PER_STEP == 0:
            #Save embedding
            temp_img = bh.read_img('temp.png')
            temp_img = np.expand_dims(temp_img, axis=0)
            temp_img = embedded_model.predict(temp_img)[0]
            embeddings.append(temp_img)
            
            #Select what buttons to press
            temp_action = np.random.choice(4096, p=distribution)
            action = "{0:b}".format(temp_action).rjust(12, '0')[::-1]
            
            for j, name in enumerate(button_names):
                if action[j] == '1':
                    code += b'buttons["' + str.encode(name) + b'"] = 1;'
            
        #Press buttons
        code += b'joypad.set(buttons, 1);'
        code += b'emu.frameadvance();'
        
        bh.send(code)
        
chaos_embeddings = np.array(embeddings)
os.remove('temp.png')

# Visualization

In [ ]:
#Plot bounding box sum and nuclear norm as a function of time

attract_bbox_sum, attract_nuc_norm = get_score(attract_embeddings)
chaos_bbox_sum, chaos_nuc_norm = get_score(chaos_embeddings)

plot(attract_bbox_sum, label='Attract Mode')
plot(chaos_bbox_sum, label='Chaos Monkey')
title('Bounding Box Sum')
legend()
show()

figure()
plot(attract_nuc_norm, label='Attract Mode')
plot(chaos_nuc_norm, label='Chaos Monkey')
title('Nuclear Norm')
legend()
show()

In [ ]:
#Plot attract mode data and chaos monkey data with tSNE

tsne = TSNE(n_components=2)
tsne.fit(np.concatenate((attract_embeddings, chaos_embeddings)))

attract_tsne = tsne.fit_transform(attract_embeddings)
chaos_tsne = tsne.fit_transform(chaos_embeddings)

scatter(attract_tsne[:, 0], attract_tsne[:, 1], color='r', label='Attract Mode')
scatter(chaos_tsne[:, 0], chaos_tsne[:, 1], color='b', label='Chaos Monkey')
legend()
show()